In [1]:
# CELL 1: importy i konfiguracja
import os, re, math, json, gc
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Deterministyczność (opcjonalnie)
SEED = 42
tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)


In [2]:
import pandas as pd
import numpy as np
from difflib import get_close_matches

PATH = 'Data_state_LSTM_predicted_full_v4_FINAL.csv'

# 1) Wczytaj bez nagłówka (plik z LSTM ma separator ';')
df_raw = pd.read_csv(PATH, sep=';', encoding='utf-8-sig', header=None)
print(f'Wczytano (bez nagłówka) {df_raw.shape} z: {PATH}')

# 2) Znajdź wiersz nagłówka: szukamy takiego, który zawiera tokeny przewidywanych kolumn
header_tokens = {'predict_state', 'predicted_state', 'predict_loc', 'predicted_loc', 'saleid', 'title', 'description'}
header_idx = None
for i in range(min(50, len(df_raw))):  # skan 50 pierwszych wierszy
    row_vals = df_raw.iloc[i].astype(str).str.lower().str.strip()
    if any(tok in set(row_vals.values) for tok in header_tokens):
        header_idx = i
        break

if header_idx is None:
    # fallback: jeśli nie wykryto, spróbuj użyć pierwszego wiersza jako nagłówka
    header_idx = 0

# 3) Ustaw nagłówek z wybranego wiersza i usuń go z danych
new_columns = df_raw.iloc[header_idx].astype(str).tolist()
df = df_raw.drop(index=header_idx).reset_index(drop=True)
df.columns = new_columns
print('Nagłówek ustawiony na podstawie wiersza:', header_idx)
print('Kolumny (po korekcie nagłówka):', list(df.columns)[:20])

# 4) Normalizacja nazw i mapowanie kolumn lokalizacji/stanu budynku
def canon(s: str) -> str:
    s = str(s)
    return ''.join(ch for ch in s.lower() if ch.isalnum() or ch == '_')

col_map = {canon(c): c for c in df.columns}

loc_candidates   = ['predicted_loc', 'predict_loc', 'predictedloc', 'predictloc']
state_candidates = ['predict_state', 'predicted_state', 'predictstate', 'predictedstate']

def resolve(col_map, candidates):
    # najpierw próbuj pełnych kandydatów
    for cand in candidates:
        key = canon(cand)
        if key in col_map:
            return col_map[key]
    # fuzzy podpowiedzi (weź pierwszy kandydat jako wzorzec)
    base = canon(candidates)
    close = get_close_matches(base, list(col_map.keys()), n=3, cutoff=0.6)
    hints = [col_map[k] for k in close]
    return None, hints

def unpack(res):
    if isinstance(res, tuple):
        return None, res[1]
    return res, []

loc_res = resolve(col_map, loc_candidates)
state_res = resolve(col_map, state_candidates)
loc_name, loc_hints     = unpack(loc_res)
state_name, state_hints = unpack(state_res)

if loc_name is None:
    raise ValueError(f'Nie znaleziono kolumny lokalizacji (szukano: {loc_candidates}). '
                     f'Podobne: {loc_hints}')
if state_name is None:
    raise ValueError(f'Nie znaleziono kolumny stanu (szukano: {state_candidates}). '
                     f'Podobne: {state_hints}')

# 5) Przemianuj na kanoniczne nazwy używane dalej w pipeline
df = df.rename(columns={loc_name: 'Predicted_Loc', state_name: 'Predict_State'})
df['Predicted_Loc'] = df['Predicted_Loc'].astype(str).fillna('Brak Danych')
df['Predict_State'] = df['Predict_State'].astype(str).fillna('Brak Danych')

print('Gotowe nazwy:', {'Predicted_Loc': loc_name, 'Predict_State': state_name})


Wczytano (bez nagłówka) (1467263, 56) z: Data_state_LSTM_predicted_full_v4_FINAL.csv
Nagłówek ustawiony na podstawie wiersza: 0
Kolumny (po korekcie nagłówka): ['88', 'nan', '14', 'Mieszkanie trzypokojowe na sprzedaż', 'Mieszkanie o powierzchni 73m2 znajduje się na 3 piętrze bloku z cegły z 2005 roku, w doskonałej lokalizacji osiedla Bojary, zaledwie 2 minuty od Pałacu Branickich, co sprawia, że jest to ścisłe centrum miasta. Mieszkanie zostało urządzone w swojskim klimacie, co czyni je wyjątkowym i wyróżnia się spośród innych dostępnych na rynku. Wejście do mieszkania jest wygodne i szerokie, a dodatkowo została zamontowana szafa wnękowa na okrycia wierzchnie, co zapewnia praktyczne rozwiązanie na przechowywanie ubrań i innych przedmiotów. Zabudowa kuchenna w mieszkaniu jest wyjątkowo zaakcentowana i wykonana z płytek z glazury, co nadaje jej wiejski klimat. Jest to unikalne rozwiązanie, które wyróżnia się spośród innych standardowych kuchni. Dodatkowo, zabudowa kuchenna utrzymana jes

ValueError: Nie znaleziono kolumny lokalizacji (szukano: ['predicted_loc', 'predict_loc', 'predictedloc', 'predictloc']). Podobne: []

In [ ]:
# CELL 3: inżynieria cech (liczbowe + porządkowe)
df_proc = df.copy()

# Cechy liczbowe
num_cols = ['Area','NumberOfRooms','Floor','Floors']
for c in num_cols:
    if c in df_proc.columns:
        df_proc[c] = pd.to_numeric(df_proc[c], errors='coerce')

# BuiltYear -> BuildingAge
if 'BuiltYear' in df_proc.columns:
    by = pd.to_numeric(df_proc['BuiltYear'], errors='coerce')
    median_year = by.dropna().median() if not np.isnan(by.dropna().median()) else 2000
    by = by.fillna(median_year)
    year_int = by.astype(int)
    current_year = datetime.now().year
    df_proc['BuildingAge'] = (current_year - year_int).clip(lower=0, upper=200)
else:
    df_proc['BuildingAge'] = np.nan

# docelowe kolumny liczbowe do modelu
numeric_features = ['Area','NumberOfRooms','Floor','Floors','BuildingAge']

# Cechy kategoryczne (priorytetowe)
cat_priority = ['Predict_State','Predicted_Loc']

# Dodatkowe kategorie (opcjonalnie, jeśli istnieją)
cat_optional = [c for c in ['BuildingType','TypeOfMarket','Type','OfferFrom','OwnerType'] if c in df_proc.columns]

categorical_features = cat_priority + cat_optional

# Czyszczenie braków
for c in categorical_features:
    df_proc[c] = df_proc[c].astype(str).fillna('unknown').replace({'nan':'unknown','None':'unknown'})

# Target
df_proc['Price'] = pd.to_numeric(df_proc['Price'], errors='coerce')
df_proc = df_proc.dropna(subset=['Price','Area'])  # minimalne wymagania
print('Dane po wstępnym przetwarzaniu:', df_proc.shape)


In [ ]:
# CELL 4: split + tf.data
from sklearn.model_selection import train_test_split

features = numeric_features + categorical_features
target = 'Price'

train_df, val_df = train_test_split(df_proc[features + [target]], test_size=0.2, random_state=SEED)

def df_to_ds(frame, shuffle=False, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices((
        {col: frame[col].values for col in features},
        frame[target].values.astype('float32')
    ))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(frame), seed=SEED)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = df_to_ds(train_df, shuffle=True)
val_ds   = df_to_ds(val_df, shuffle=False)


In [ ]:
# CELL 5: preprocessing layers
# Normalization dla liczb
normalizers = {}
for c in numeric_features:
    norm = layers.Normalization(axis=None)
    x = train_df[c].values.astype('float32').reshape(-1,1)
    norm.adapt(x)
    normalizers[c] = norm

# StringLookup + Embedding dla kategorii
lookup_layers = {}
embed_layers  = {}
embed_dims = {}

def make_string_lookup(name, train_series):
    lk = layers.StringLookup(output_mode='int', num_oov_indices=1)
    lk.adapt(train_series.astype(str).values)
    return lk

def suggest_embed_dim(cardinality):
    # prosta heurystyka
    return int(min(50, max(4, round(np.sqrt(cardinality)))))

for c in categorical_features:
    lk = make_string_lookup(c, train_df[c])
    vocab_size = lk.vocabulary_size()
    dim = suggest_embed_dim(vocab_size)
    emb = layers.Embedding(input_dim=vocab_size+1, output_dim=dim)  # +1 zapasowo
    lookup_layers[c] = lk
    embed_layers[c]  = emb
    embed_dims[c]    = dim

print({c: (lookup_layers[c].vocabulary_size(), embed_dims[c]) for c in categorical_features})


In [ ]:
# CELL 6: model Keras (regresja wide & deep)
inputs = {}
num_tensors = []
for c in numeric_features:
    inp = layers.Input(shape=(1,), name=c, dtype=tf.float32)
    inputs[c] = inp
    x = normalizers[c](inp)
    num_tensors.append(x)

cat_tensors = []
for c in categorical_features:
    inp = layers.Input(shape=(1,), name=c, dtype=tf.string)
    inputs[c] = inp
    idx = lookup_layers[c](inp)
    emb = embed_layers[c](idx)
    emb = layers.Reshape((embed_dims[c],))(emb)  # (batch, dim)
    cat_tensors.append(emb)

# Fuzja
x = layers.Concatenate()(num_tensors + cat_tensors)

# Głowica regresyjna
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, name='price')(x)

model = keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='mse',
              metrics=[keras.metrics.MeanAbsoluteError(name='mae'),
                       keras.metrics.RootMeanSquaredError(name='rmse')])

model.summary()


In [ ]:
# CELL 7: trening
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)
csv = keras.callbacks.CSVLogger('training_regression_keras.csv', append=False)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[es, rlr, csv]
)


In [ ]:
# CELL 8: ocena + zapis
eval_res = model.evaluate(val_ds, verbose=0)
print({'loss_mse': eval_res, 'mae': eval_res[1], 'rmse': eval_res[13]})

SAVE_PATH = 'price_regressor_wide_deep.keras'
model.save(SAVE_PATH)
print(f'Zapisano model: {SAVE_PATH}')


In [ ]:
# CELL 9: inferencja na nowych danych
def predict_on_csv(path, batch_size=2048):
    df_new = pd.read_csv(path, sep=None, engine='python', low_memory=False)
    # wymagane kolumny
    need = set(numeric_features + categorical_features)
    missing = [c for c in need if c not in df_new.columns]
    for c in missing:
        if c in categorical_features:
            df_new[c] = 'unknown'
        else:
            df_new[c] = np.nan

    # building age jeżeli trzeba
    if 'BuildingAge' in numeric_features and 'BuildingAge' not in df_new.columns:
        if 'BuiltYear' in df_new.columns:
            by = pd.to_numeric(df_new['BuiltYear'], errors='coerce')
            med = by.dropna().median() if not np.isnan(by.dropna().median()) else 2000
            by = by.fillna(med).astype(int)
            df_new['BuildingAge'] = (datetime.now().year - by).clip(lower=0, upper=200)
        else:
            df_new['BuildingAge'] = np.nan

    # typy
    for c in numeric_features:
        df_new[c] = pd.to_numeric(df_new[c], errors='coerce')
    for c in categorical_features:
        df_new[c] = df_new[c].astype(str).fillna('unknown').replace({'nan':'unknown','None':'unknown'})

    ds = tf.data.Dataset.from_tensor_slices({col: df_new[col].values for col in numeric_features + categorical_features})
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    preds = model.predict(ds, verbose=0).reshape(-1)
    out = df_new.copy()
    out['PredictedPrice'] = preds
    return out

# przykład:
# preds_df = predict_on_csv('Data_state_LSTM_predicted_full_v4_FINAL.csv')
# preds_df[['SaleId','Price','Predicted_Loc','Predict_State','PredictedPrice']].head(10)
